In [15]:
import numpy as np
import  matplotlib.pyplot as plt
import cv2 as cv
from PIL import Image
from patchify import patchify

In [16]:
image = cv.imread("./Final Dataset/train/River/River_1.jpg")
image = np.asarray(image)
print(image.shape)
patches = patchify(image,(16,16,3),step=16)
print(patches.shape)

(64, 64, 3)
(4, 4, 1, 16, 16, 3)


In [17]:
image_patch_size = 16

In [18]:
image_dataset = []

size_x = (image.shape[1]//image_patch_size)*image_patch_size
size_y = (image.shape[0]//image_patch_size)*image_patch_size
image = Image.fromarray(image)
print(image.show())
image = image.crop((0,0, size_x, size_y))
image = np.array(image)
patched_images = patchify(image, (image_patch_size, image_patch_size, 3), step=image_patch_size)
print(patched_images.shape)
for i in range(patched_images.shape[0]):
  for j in range(patched_images.shape[1]):
    individual_patched_image = patched_images[i,j,:,:]
    individual_patched_image = individual_patched_image[0]
    image_dataset.append(individual_patched_image)
    image = Image.fromarray(individual_patched_image)
    print(image.show())

None
(4, 4, 1, 16, 16, 3)


In [23]:
import torch
import torchvision.transforms.functional as TF

def generate_positive_patches(image, object_bbox, patch_size=64, iou_threshold=0.5):
    patches = []
    for bbox in object_bbox:
        x, y, w, h = bbox
        # Crop patch around the object
        patch = TF.crop(image, y, x, h, w)
        # Compute IoU
        intersection = torch.tensor([max(0, min(x+w, x2+w2) - max(x, x2)) * max(0, min(y+h, y2+h2) - max(y, y2)) for x2, y2, w2, h2 in object_bbox])
        union = w * h + torch.tensor([(w2 * h2) for _, _, w2, h2 in object_bbox]) - intersection
        iou = intersection / union
        # Label patch based on IoU
        label = torch.tensor([1 if iou_val >= iou_threshold else 0 for iou_val in iou])
        patches.append((patch, label))
    return patches

def generate_negative_patches(image, object_bbox, num_patches=5, patch_size=64):
    patches = []
    height, width = image.shape[-2:]
    for _ in range(num_patches):
        # Randomly crop patch from background
        x = torch.randint(0, width - patch_size, (1,))
        y = torch.randint(0, height - patch_size, (1,))
        patch = TF.crop(image, int(y), int(x), patch_size, patch_size)
        patches.append((patch, torch.tensor([0])))
    return patches

# Example usage
image = torch.rand(3, 256, 256)  # Example image tensor (3 channels, 256x256)
object_bbox = [(50, 50, 100, 100)]  # Example object bounding box (x, y, width, height)

positive_patches = generate_positive_patches(image, object_bbox)
negative_patches = generate_negative_patches(image, object_bbox)

# Now you have positive_patches and negative_patches with corresponding labels


In [29]:
import torch
from torchvision.transforms.functional import to_pil_image
from PIL import Image
import matplotlib.pyplot as plt

def display_patches(patches):
    for i, (patch, _) in enumerate(patches):
        patch_image = to_pil_image(patch)
        patch_image.show()

# Example usage
image = cv.imread("./Final Dataset/train/River/River_1.jpg") # Example image tensor (3 channels, 256x256)
image = Image.fromarray(image)
object_bbox = [(50, 50, 100, 100)]  # Example object bounding box (x, y, width, height)

positive_patches = generate_positive_patches(image, object_bbox)
negative_patches = generate_negative_patches(image, object_bbox)

print("Displaying Positive Patches:")
display_patches(positive_patches)

print("Displaying Negative Patches:")
display_patches(negative_patches)


AttributeError: 'Image' object has no attribute 'shape'